In [1]:
import torch
import tiktoken
import os

from gpt_model import GPTModel
from data_loader_v1 import create_dataloader_v1
from generate_text import generate

### Detect if GPU is available

In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")

print(f"Using {device} device.")

Using cuda device.


### Set up model configuration 

In [26]:
GPT_CONFIG_124M = {
    "vocab_size": 10000,    # Vocabulary size
    "context_length": 256,  # Context length
    "emb_dim": 768,         # Embedding dimension
    "n_heads": 6,          # Number of attention heads
    "n_layers": 8,         # Number of layers
    "drop_rate": 0.3,       # Dropout rate
    "qkv_bias": True,      # Query-Key-Value bias
    "device": device,
}

In [4]:
import sentencepiece as spm

spm.SentencePieceTrainer.train(
    input='all_books.txt',
    model_prefix='gpt_custom_tokenizer',
    vocab_size=GPT_CONFIG_124M["vocab_size"],
    model_type='bpe',         # You can also try 'unigram'
    character_coverage=1.0,   # Keeps all characters (good for English)
    hard_vocab_limit=False,
    bos_id=-1,                # Optional: GPT usually doesn't need BOS token
    eos_id=-1                 # Optional: EOS handled manually in GPT
);

sentencepiece_trainer.cc(78) LOG(INFO) Starts training with : 
trainer_spec {
  input: all_books.txt
  input_format: 
  model_prefix: gpt_custom_tokenizer
  model_type: BPE
  vocab_size: 10000
  self_test_sample_size: 0
  character_coverage: 1
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  seed_sentencepieces_file: 
  hard_vocab_limit: 0
  use_all_vocab: 0
  unk_id: 0
  bos_id: -1
  eos_id: -1
  pad_id: -1
  unk_piece: <unk>
  bos_piece: <s>
  eos_piece: </s>
  pad_piece: <pad>
  unk_surface:  ⁇ 
  enable_differential_privacy: 0


### Load training and validation data files

In [5]:
train_file_path = 'train_text_data.txt'
val_file_path = 'val_text_data.txt'

with open(train_file_path, "r", encoding="utf-8") as file:
    train_data = file.read()
with open(val_file_path, "r", encoding="utf-8") as file:
    val_data = file.read()

LOG(INFO) Added: freq=9 size=7780 all=24536 active=1168 piece=▁officious
bpe_model_trainer.cc(268) LOG(INFO) Added: freq=9 size=7800 all=24517 active=1149 piece=▁expediency
bpe_model_trainer.cc(159) LOG(INFO) Updating active symbols. max_freq=9 min_freq=6
bpe_model_trainer.cc(268) LOG(INFO) Added: freq=9 size=7820 all=24499 active=1208 piece=▁humiliation
bpe_model_trainer.cc(268) LOG(INFO) Added: freq=8 size=7840 all=24483 active=1192 piece=Al
bpe_model_trainer.cc(268) LOG(INFO) Added: freq=8 size=7860 all=24517 active=1226 piece=spe
bpe_model_trainer.cc(268) LOG(INFO) Added: freq=8 size=7880 all=24536 active=1245 piece=itia
bpe_model_trainer.cc(268) LOG(INFO) Added: freq=8 size=7900 all=24558 active=1267 piece=▁ecl
bpe_model_trainer.cc(159) LOG(INFO) Updating active symbols. max_freq=8 min_freq=6
bpe_model_trainer.cc(268) LOG(INFO) Added: freq=8 size=7920 all=24583 active=1251 piece=isons
bpe_model_trainer.cc(268) LOG(INFO) Added: freq=8 size=7940 all=24603 active=1271 piece=���Take
b

### Initialize data loaders for training
Data loaders implementation can be found in `./data_loader_v1.py`.

This implementation follows the omplementation detailed in _Raschka, Sebastian. Build a Large Language Model (From Scratch). Manning Publications, 2024_

In [6]:
train_ratio = 0.90

train_loader = create_dataloader_v1(
    train_data,
    batch_size=16,
    max_length=GPT_CONFIG_124M["context_length"],
    stride=GPT_CONFIG_124M["context_length"],
    drop_last=True,
    shuffle=True,
    num_workers=0
)

val_loader = create_dataloader_v1(
    val_data,
    batch_size=16,
    max_length=GPT_CONFIG_124M["context_length"],
    stride=GPT_CONFIG_124M["context_length"],
    drop_last=False,
    shuffle=False,
    num_workers=0
)

In [7]:
import gc

def clean(): 
    """
    This is a function for GPU data claening before and after training
    """
    
    os.environ["PYTORCH_MPS_HIGH_WATERMARK_RATIO"] = "0.0"
    
    gc.collect()  # Force garbage collection
    torch.mps.empty_cache()  # Attempt to release MPS memory
    
    # Move tensors to CPU
    for tensor in list(globals().values()):
        if isinstance(tensor, torch.Tensor) and tensor.device == torch.device("mps"):
            tensor.to("cpu")

    # Delete all tensors
    del tensor
    torch.mps.empty_cache()
    gc.collect()  # Force garbage collection
    print("MPS Available:", torch.backends.mps.is_available())
    print("Allocated Memory:", torch.mps.current_allocated_memory() / (1024**2), "MB")

# Training

In [27]:
from pre_train import train_model_simple
import time

train_losses, val_losses, track_tokens_seen = [], [], []

def train(train_loader, val_loader,
          num_epochs=10, eval_iter=5, 
          sample_text="It is a truth universally acknowledged, that a single man in possession of a good fortune, must be",
          checkpoint_path="model_and_optimizer.pth"):

    global train_losses, val_losses, track_tokens_seen  # Ensure these are updated globally

    if device == "mps":
        clean()
        print(50 * "=")
        print("Starting training...")
    if device == "cuda":
        torch.cuda.empty_cache()
        torch.cuda.memory_summary()
        print(50 * "=")
        print("Starting training...")

    start_time = time.time()

    torch.manual_seed(123)
    model = GPTModel(GPT_CONFIG_124M)
    model.to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=0.0002, betas=(0.9, 0.98), weight_decay=0.01)

    # Pass train_losses and val_losses as references
    train_model_simple(
        model, train_loader, val_loader, optimizer,
        num_epochs=num_epochs, eval_iter=eval_iter,
        start_context=sample_text, cfg=GPT_CONFIG_124M,
        checkpoint_path=checkpoint_path,
        train_losses=train_losses, val_losses=val_losses,
        track_tokens_seen=track_tokens_seen
    )
    
    end_time = time.time()
    execution_time_minutes = (end_time - start_time) / 60
    print(f"Training completed in {execution_time_minutes:.2f} minutes.")
    
    if device == "mps":
        print(50 * "=")
        clean()
    if device == "cuda":
        print(50 * "=")
        torch.cuda.empty_cache()
        torch.cuda.memory_summary()
    
    return model

In [28]:
gc.collect()  # Force garbage collection

3974

### Train the model on training data

In [29]:
# train model on all works

train(train_loader, val_loader, num_epochs=10,
      eval_iter=5, sample_text="He inherited the estate",
      checkpoint_path="model_and_optimizer_best_.pth");

Ep 1 (Step 000000): Train loss 8.667, Val loss 8.638
Ep 1 (Step 000005): Train loss 7.432, Val loss 7.388
Ep 1 (Step 000010): Train loss 6.888, Val loss 6.858
Ep 1 (Step 000015): Train loss 6.549, Val loss 6.547
Ep 1 (Step 000020): Train loss 6.389, Val loss 6.408
Ep 1 (Step 000025): Train loss 6.363, Val loss 6.367
Ep 1 (Step 000030): Train loss 6.285, Val loss 6.374
Ep 1 (Step 000035): Train loss 6.222, Val loss 6.303
Ep 1 (Step 000040): Train loss 6.153, Val loss 6.222


KeyboardInterrupt: 

### Load trained model

In [18]:
model = GPTModel(GPT_CONFIG_124M)
model.to("cpu")
optimizer = torch.optim.AdamW(model.parameters(), lr=0.0004, weight_decay=0.1)

checkpoint = torch.load("model_and_optimizer_5.pth", weights_only=True)
model.load_state_dict(checkpoint["model_state_dict"])
optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
model.eval();

In [19]:
from generate_text import generate

torch.set_printoptions(profile="full")
text = generate(
    model=model, prompt="Miss Bennet has inherited the estate from her aunt, so she must",
    max_new_tokens=50, context_size=GPT_CONFIG_124M['context_length'],
    device="cpu",
    temperature=0.7,
    top_k=50
)

splitted = text.split("\n")
for txt in splitted:
    print(txt)
    
print(50*"=")
    
text = generate(
    model=model, prompt="Mr. Darcy has inherited the estate from his aunt, so he must",
    max_new_tokens=50, context_size=GPT_CONFIG_124M['context_length'],
    device="cpu",
    temperature=0.7,
    top_k=50,
)

splitted = text.split("\n")
for txt in splitted:
    print(txt)

Miss Bennet has inherited the estate from her aunt, so she must have been the subject, and so much in love with her, as to her that Mr. and Mrs. Gardiner, that you should not be so disappointed, that his absence is not to be in his way. There is a very young man
Mr. Darcy has inherited the estate from his aunt, so he must have done so much in all his own. It is not a very handsome young man, that is, I hope, to be very glad to be so in my power to be in the least. I am very glad that I hope it will be


In [26]:
from generate_text import generate

torch.set_printoptions(profile="full")
text = generate(
    model=model, prompt="A wife is",
    max_new_tokens=30, context_size=GPT_CONFIG_124M['context_length'],
    device="cpu",
    temperature=0.5,
    top_k=40
)

splitted = text.split("\n")
for txt in splitted:
    print(txt)
    
print(50*"=")
    
text = generate(
    model=model, prompt="A husband is",
    max_new_tokens=30, context_size=GPT_CONFIG_124M['context_length'],
    device="cpu",
    temperature=0.5,
    top_k=40,
)

splitted = text.split("\n")
for txt in splitted:
    print(txt)

A wife is a young man, and is not a very agreeable young man, and the other people know. But it is a very different sort of man, and
A husband is a man, and is a very pretty young man, and a very agreeable man, and a very good young man, and a very good man,


In [24]:
from generate_text import generate

torch.set_printoptions(profile="full")
text = generate(
    model=model, prompt="A good lady ought to be",
    max_new_tokens=30, context_size=GPT_CONFIG_124M['context_length'],
    device="cpu",
    temperature=0.7,
    top_k=30
)

splitted = text.split("\n")
for txt in splitted:
    print(txt)
    
print(50*"=")
    
text = generate(
    model=model, prompt="A highly respectable man ought to be",
    max_new_tokens=30, context_size=GPT_CONFIG_124M['context_length'],
    device="cpu",
    temperature=0.7,
    top_k=30,
)

splitted = text.split("\n")
for txt in splitted:
    print(txt)

A good lady ought to be a very desirable thing. The two girls are a man, and every body ought to be allowed to be the best, and all must be a little
A highly respectable man ought to be in town of him."
"I am sure," cried Catherine, "for I always felt that I am so kind as to the match as possible


In [16]:
if device == "mps":
    clean()